# Advanced Strategy Components Tutorial

This tutorial covers **Phase 2** of SignalFlow's strategy components:

1. **Position Sizing** - Flexible capital allocation algorithms
2. **Entry Filters** - Pre-trade validation to improve signal quality
3. **Signal Aggregation** - Combine signals from multiple detectors

### Architecture

```
Signals (from detectors)
    |
    v
[SignalAggregator] -----> Aggregated Signals
    |
    v
SignalEntryRule.check_entries()
    |
    +---> [EntryFilter(s)] -----> allow_entry() -> bool
    |
    +---> [PositionSizer] -----> compute_size() -> notional
    |
    v
Order with computed qty
```

### Table of Contents

1. [Setup](#1-setup)
2. [Position Sizing Strategies](#2-position-sizing-strategies)
3. [Entry Filters](#3-entry-filters)
4. [Signal Aggregation](#4-signal-aggregation)
5. [Integration with SignalEntryRule](#5-integration-with-signalentryrule)
6. [Grid Trading Example](#6-grid-trading-example)
7. [Full Backtest Example](#7-full-backtest-example)

## 1. Setup

In [ ]:
from datetime import datetime
from pathlib import Path

import polars as pl
import signalflow as sf

# Position sizers
from signalflow.strategy.component.sizing import (
    FixedFractionSizer,
    KellyCriterionSizer,
    MartingaleSizer,
    RiskParitySizer,
    SignalContext,
    SignalStrengthSizer,
    VolatilityTargetSizer,
)

# Entry filters
from signalflow.strategy.component.entry import (
    CompositeEntryFilter,
    CorrelationFilter,
    DrawdownFilter,
    PriceDistanceFilter,
    RegimeFilter,
    SignalAccuracyFilter,
    TimeOfDayFilter,
    VolatilityFilter,
)

# Signal aggregation
from signalflow.strategy.component.entry import SignalAggregator, VotingMode

# Core components
from signalflow.core import (
    Position,
    PositionType,
    Signals,
    SignalType,
    StrategyState,
)


### Helper: Create Test Environment

We'll create a reusable strategy state and signal context for testing components.

In [ ]:
def create_test_state(cash: float = 10000.0) -> StrategyState:
    """Create a test strategy state with specified cash."""
    state = StrategyState(strategy_id="tutorial")
    state.portfolio.cash = cash
    return state


def create_signal_context(
    pair: str = "BTCUSDT",
    signal_type: str = "rise",
    probability: float = 0.8,
    price: float = 50000.0,
    timestamp: datetime = None,
) -> SignalContext:
    """Create a test signal context."""
    return SignalContext(
        pair=pair,
        signal_type=signal_type,
        probability=probability,
        price=price,
        timestamp=timestamp or datetime(2024, 1, 1, 10, 0),
    )


# Test it
state = create_test_state(10000.0)
signal = create_signal_context()
prices = {"BTCUSDT": 50000.0, "ETHUSDT": 3000.0, "SOLUSDT": 100.0}

print(f"State: cash=${state.portfolio.cash:,.2f}")
print(f"Signal: {signal.pair} {signal.signal_type} (p={signal.probability}) @ ${signal.price:,.2f}")

State: cash=$10,000.00
Signal: BTCUSDT rise (p=0.8) @ $50,000.00


## 2. Position Sizing Strategies

Position sizers determine **how much** capital to allocate to each trade. All sizers implement:

```python
def compute_size(signal: SignalContext, state: StrategyState, prices: dict[str, float]) -> float:
    """Return notional value (quote currency). 0.0 to skip."""
```

Available sizers:

| Sizer | Strategy | Key Params |
|-------|----------|------------|
| `FixedFractionSizer` | Fixed % of equity | `fraction`, `min_notional`, `max_notional` |
| `SignalStrengthSizer` | Scale by probability | `base_size`, `min_probability` |
| `KellyCriterionSizer` | Optimal f* formula | `kelly_fraction`, `min_trades_for_stats` |
| `VolatilityTargetSizer` | Inverse volatility | `target_volatility`, `default_volatility_pct` |
| `RiskParitySizer` | Equal risk budget | `target_positions`, `default_volatility_pct` |
| `MartingaleSizer` | Grid/DCA scaling | `base_size`, `multiplier`, `max_grid_levels` |

### 2.1 FixedFractionSizer

The simplest sizer: allocate a fixed percentage of portfolio equity to each trade.

In [ ]:
# Basic usage: 2% of equity per trade
sizer = FixedFractionSizer(fraction=0.02)
size = sizer.compute_size(signal, state, prices)
print(f"2% of $10,000 = ${size:,.2f}")

# With min/max constraints
sizer_constrained = FixedFractionSizer(
    fraction=0.02,
    min_notional=50.0,   # Skip if position too small
    max_notional=500.0,  # Cap maximum size
)
size_constrained = sizer_constrained.compute_size(signal, state, prices)
print(f"With max cap: ${size_constrained:,.2f}")

# Too small - returns 0
sizer_high_min = FixedFractionSizer(fraction=0.001, min_notional=50.0)
size_skip = sizer_high_min.compute_size(signal, state, prices)
print(f"Below min threshold: ${size_skip:,.2f} (skipped)")

2% of $10,000 = $200.00
With max cap: $200.00
Below min threshold: $0.00 (skipped)


### 2.2 SignalStrengthSizer

Scale position size based on signal probability/confidence.

In [ ]:
sizer = SignalStrengthSizer(
    base_size=1000.0,       # Maximum size at probability=1.0
    min_probability=0.5,    # Skip signals below this
    max_notional=2000.0,    # Hard cap
)

# Compare different probability levels
for prob in [0.9, 0.7, 0.5, 0.3]:
    ctx = create_signal_context(probability=prob)
    size = sizer.compute_size(ctx, state, prices)
    status = "" if size > 0 else "(skipped - below min_probability)"
    print(f"Probability {prob:.0%}: ${size:,.2f} {status}")

Probability 90%: $900.00 
Probability 70%: $700.00 
Probability 50%: $500.00 
Probability 30%: $0.00 (skipped - below min_probability)


### 2.3 KellyCriterionSizer

Uses the Kelly formula to compute optimal position sizing based on edge and payoff ratio:

$$f^* = \frac{p \cdot b - q}{b}$$

Where:
- $p$ = win probability
- $q$ = loss probability (1 - p)
- $b$ = payoff ratio (avg win / avg loss)

In [ ]:
sizer = KellyCriterionSizer(
    kelly_fraction=0.5,          # Half-Kelly (more conservative)
    use_signal_probability=True, # Use signal.probability as win rate
    default_payoff_ratio=1.5,    # Expected win/loss ratio
    max_fraction=0.25,           # Never exceed 25% of equity
    min_notional=10.0,           # Minimum position size
)

# With high probability signal
ctx_high = create_signal_context(probability=0.8)
size_high = sizer.compute_size(ctx_high, state, prices)
print(f"High probability (80%): ${size_high:,.2f}")

# With low probability signal (negative Kelly -> 0)
ctx_low = create_signal_context(probability=0.3)
size_low = sizer.compute_size(ctx_low, state, prices)
print(f"Low probability (30%): ${size_low:,.2f} (negative Kelly)")

# Kelly formula breakdown:
p, b = 0.8, 1.5
q = 1 - p
kelly_f = (p * b - q) / b
half_kelly = kelly_f * 0.5
print(f"\nKelly calculation: f* = ({p}*{b} - {q})/{b} = {kelly_f:.3f}")
print(f"Half-Kelly: {half_kelly:.3f}")
print(f"Theoretical max: ${10000 * half_kelly:,.2f} (before 25% cap)")

High probability (80%): $2,500.00
Low probability (30%): $0.00 (negative Kelly)

Kelly calculation: f* = (0.8*1.5 - 0.19999999999999996)/1.5 = 0.667
Half-Kelly: 0.333
Theoretical max: $3,333.33 (before 25% cap)


### 2.4 VolatilityTargetSizer

Size positions inversely to volatility to target a constant risk level:

$$\text{notional} = \frac{\text{target\_vol} \times \text{equity}}{\text{asset\_vol}}$$

In [ ]:
sizer = VolatilityTargetSizer(
    target_volatility=0.01,      # Target 1% daily volatility exposure
    default_volatility_pct=0.02, # Fallback if no ATR data
    max_fraction=0.20,           # Max 20% of equity per position
)

# Low volatility asset -> larger position
state_low_vol = create_test_state(10000.0)
state_low_vol.runtime["atr"] = {"BTCUSDT": 250.0}  # 0.5% of price
size_low_vol = sizer.compute_size(signal, state_low_vol, prices)
print(f"Low volatility (0.5%): ${size_low_vol:,.2f}")

# High volatility asset -> smaller position  
state_high_vol = create_test_state(10000.0)
state_high_vol.runtime["atr"] = {"BTCUSDT": 2500.0}  # 5% of price
size_high_vol = sizer.compute_size(signal, state_high_vol, prices)
print(f"High volatility (5%): ${size_high_vol:,.2f}")

# No ATR data -> uses default volatility
size_default = sizer.compute_size(signal, state, prices)
print(f"No ATR (default 2%): ${size_default:,.2f}")

Low volatility (0.5%): $2,000.00
High volatility (5%): $2,000.00
No ATR (default 2%): $2,000.00


### 2.5 RiskParitySizer

Allocate equal risk budget across a target number of positions.

In [ ]:
sizer = RiskParitySizer(
    target_positions=10,         # Expect to hold ~10 positions
    default_volatility_pct=0.02, # Fallback volatility
)

# With volatility data
state_vol = create_test_state(10000.0)
state_vol.runtime["atr"] = {"BTCUSDT": 500.0}  # 1% of price

size = sizer.compute_size(signal, state_vol, prices)
print(f"Risk parity size: ${size:,.2f}")
print(f"Max per position: ${10000/10:,.2f} (equity / target_positions)")

Risk parity size: $1,000.00
Max per position: $1,000.00 (equity / target_positions)


### 2.6 MartingaleSizer

For grid/DCA strategies: increase position size with each grid level.

In [ ]:
sizer = MartingaleSizer(
    base_size=100.0,      # First entry size
    multiplier=1.5,       # Each level is 1.5x previous
    max_grid_levels=5,    # Max 5 entries per pair
    max_notional=1000.0,  # Hard cap per entry
)

# Simulate adding grid levels
for level in range(6):
    state_grid = create_test_state(10000.0)
    
    # Add existing positions for this pair
    for i in range(level):
        pos = Position(
            pair="BTCUSDT",
            position_type=PositionType.LONG,
            entry_price=50000.0 - i * 1000,
            qty=0.002,
        )
        state_grid.portfolio.positions[f"pos_{i}"] = pos
    
    size = sizer.compute_size(signal, state_grid, prices)
    expected = 100 * (1.5 ** level) if level < 5 else 0
    status = "" if size > 0 else "(max levels reached)"
    print(f"Level {level}: ${size:,.2f} {status}")

Level 0: $100.00 
Level 1: $150.00 
Level 2: $225.00 
Level 3: $337.50 
Level 4: $506.25 
Level 5: $0.00 (max levels reached)


## 3. Entry Filters

Entry filters validate signals before opening positions. All filters implement:

```python
def allow_entry(signal: SignalContext, state: StrategyState, prices: dict[str, float]) -> tuple[bool, str]:
    """Return (allowed, reason). Empty reason if allowed."""
```

Available filters:

| Filter | Blocks When | Key Params |
|--------|-------------|------------|
| `RegimeFilter` | Signal doesn't match market regime | N/A |
| `VolatilityFilter` | Volatility outside min/max range | `min_volatility`, `max_volatility` |
| `DrawdownFilter` | Drawdown exceeds threshold | `max_drawdown`, `recovery_threshold` |
| `CorrelationFilter` | Too many correlated positions | `max_correlation`, `max_correlated_positions` |
| `TimeOfDayFilter` | Outside trading hours | `allowed_hours`, `blocked_hours` |
| `PriceDistanceFilter` | Price too close to existing entry | `min_distance_pct`, `direction_aware` |
| `SignalAccuracyFilter` | Detector accuracy below threshold | `min_accuracy`, `min_samples` |
| `CompositeEntryFilter` | Combines multiple filters | `filters`, `require_all` |

### 3.1 RegimeFilter

Only allow signals that match the current market regime.

In [ ]:
filter_ = RegimeFilter()

# Test different regimes
regimes = ["trend_up", "trend_down", "mean_reversion_oversold", "choppy"]

for regime in regimes:
    state_regime = create_test_state()
    state_regime.runtime["regime"] = {"BTCUSDT": regime}
    
    # RISE signal
    ctx_rise = create_signal_context(signal_type="rise")
    allowed, reason = filter_.allow_entry(ctx_rise, state_regime, prices)
    print(f"Regime: {regime:25} | RISE signal: {'ALLOWED' if allowed else 'BLOCKED'} {reason}")

Regime: trend_up                  | RISE signal: ALLOWED 
Regime: trend_down                | RISE signal: BLOCKED regime=trend_down not in ['trend_up', 'mean_reversion_oversold']
Regime: mean_reversion_oversold   | RISE signal: ALLOWED 
Regime: choppy                    | RISE signal: BLOCKED regime=choppy not in ['trend_up', 'mean_reversion_oversold']


### 3.2 VolatilityFilter

Skip trading when volatility is too low (no opportunity) or too high (too risky).

In [ ]:
filter_ = VolatilityFilter(
    min_volatility=0.005,  # Need at least 0.5% vol
    max_volatility=0.03,   # Max 3% vol
)

# Test different volatility levels
for atr_pct in [0.002, 0.01, 0.05]:
    atr = signal.price * atr_pct
    state_vol = create_test_state()
    state_vol.runtime["atr"] = {"BTCUSDT": atr}
    
    allowed, reason = filter_.allow_entry(signal, state_vol, prices)
    status = "ALLOWED" if allowed else f"BLOCKED: {reason}"
    print(f"ATR {atr_pct:.1%}: {status}")

ATR 0.2%: BLOCKED: volatility=0.0020 < min=0.005
ATR 1.0%: ALLOWED
ATR 5.0%: BLOCKED: volatility=0.0500 > max=0.03


### 3.3 DrawdownFilter

Pause trading after significant drawdown until recovery.

In [ ]:
filter_ = DrawdownFilter(
    max_drawdown=0.10,         # Pause at 10% drawdown
    recovery_threshold=0.05,   # Resume when back to 5%
)

# Simulate drawdown cycle
drawdown_levels = [0.05, 0.12, 0.08, 0.04, 0.06]

for dd in drawdown_levels:
    state_dd = create_test_state()
    state_dd.metrics["current_drawdown"] = dd
    
    allowed, reason = filter_.allow_entry(signal, state_dd, prices)
    status = "ALLOWED" if allowed else f"BLOCKED: {reason}"
    print(f"Drawdown {dd:.0%}: {status}")

Drawdown 5%: ALLOWED
Drawdown 12%: BLOCKED: drawdown=12.00% >= max=10.00%
Drawdown 8%: BLOCKED: paused, drawdown=8.00% > recovery=5.00%
Drawdown 4%: ALLOWED
Drawdown 6%: ALLOWED


### 3.4 TimeOfDayFilter

Restrict trading to specific hours.

In [ ]:
# Only trade during market hours (9 AM - 4 PM)
filter_ = TimeOfDayFilter(allowed_hours=list(range(9, 17)))

for hour in [8, 10, 16, 20]:
    ctx = create_signal_context(timestamp=datetime(2024, 1, 1, hour, 0))
    allowed, reason = filter_.allow_entry(ctx, state, prices)
    status = "ALLOWED" if allowed else f"BLOCKED: {reason}"
    print(f"Hour {hour:02d}:00 -> {status}")

Hour 08:00 -> BLOCKED: hour=8 not in allowed hours
Hour 10:00 -> ALLOWED
Hour 16:00 -> ALLOWED
Hour 20:00 -> BLOCKED: hour=20 not in allowed hours


### 3.5 PriceDistanceFilter

For grid strategies: only add to position when price has moved enough from last entry.

In [ ]:
filter_ = PriceDistanceFilter(
    min_distance_pct=0.02,   # Need 2% price move
    direction_aware=True,    # LONG wants lower price, SHORT wants higher
)

# Create state with existing position
state_pos = create_test_state()
pos = Position(
    pair="BTCUSDT",
    position_type=PositionType.LONG,
    entry_price=50000.0,
    qty=0.01,
    entry_time=datetime(2024, 1, 1, 9, 0),
)
state_pos.portfolio.positions[pos.id] = pos

# Test adding at different prices
test_prices = [50000, 49500, 49000, 48500, 51000]

print("Existing LONG @ $50,000. Testing RISE signals at:")
for p in test_prices:
    ctx = create_signal_context(signal_type="rise", price=p)
    allowed, reason = filter_.allow_entry(ctx, state_pos, {"BTCUSDT": p})
    pct_change = (p - 50000) / 50000
    status = "ALLOWED" if allowed else f"BLOCKED"
    print(f"  ${p:,} ({pct_change:+.1%}): {status}")

Existing LONG @ $50,000. Testing RISE signals at:
  $50,000 (+0.0%): BLOCKED
  $49,500 (-1.0%): BLOCKED
  $49,000 (-2.0%): ALLOWED
  $48,500 (-3.0%): ALLOWED
  $51,000 (+2.0%): BLOCKED


### 3.6 SignalAccuracyFilter

Track real-time signal accuracy and pause when detector performance drops.

In [ ]:
filter_ = SignalAccuracyFilter(
    min_accuracy=0.50,   # Require 50% accuracy
    min_samples=20,      # Need 20+ samples for reliable stats
)

# Test different accuracy scenarios
scenarios = [
    {"overall": 0.55, "samples": 50},  # Good accuracy, enough samples
    {"overall": 0.40, "samples": 50},  # Poor accuracy
    {"overall": 0.30, "samples": 10},  # Poor accuracy but not enough samples
]

for scenario in scenarios:
    state_acc = create_test_state()
    state_acc.runtime["signal_accuracy"] = {"BTCUSDT": scenario}
    
    allowed, reason = filter_.allow_entry(signal, state_acc, prices)
    status = "ALLOWED" if allowed else f"BLOCKED: {reason}"
    print(f"Accuracy {scenario['overall']:.0%}, {scenario['samples']} samples: {status}")

Accuracy 55%, 50 samples: ALLOWED
Accuracy 40%, 50 samples: BLOCKED: signal_accuracy=40.00% < min=50.00%
Accuracy 30%, 10 samples: ALLOWED


### 3.7 CompositeEntryFilter

Combine multiple filters with AND/OR logic.

In [ ]:
# All filters must pass (AND logic)
composite_all = CompositeEntryFilter(
    filters=[
        RegimeFilter(),
        DrawdownFilter(max_drawdown=0.10),
        TimeOfDayFilter(allowed_hours=list(range(9, 17))),
    ],
    require_all=True,
)

# Any filter must pass (OR logic)  
composite_any = CompositeEntryFilter(
    filters=[
        RegimeFilter(),
        DrawdownFilter(max_drawdown=0.10),
    ],
    require_all=False,
)

# Setup state where regime fails but drawdown passes
state_mixed = create_test_state()
state_mixed.runtime["regime"] = {"BTCUSDT": "trend_down"}  # Fails for RISE
state_mixed.metrics["current_drawdown"] = 0.05              # Passes

ctx = create_signal_context(signal_type="rise", timestamp=datetime(2024, 1, 1, 10, 0))

allowed_all, reason_all = composite_all.allow_entry(ctx, state_mixed, prices)
allowed_any, reason_any = composite_any.allow_entry(ctx, state_mixed, prices)

print("Scenario: regime=trend_down (fails), drawdown=5% (passes)")
print(f"\nrequire_all=True:  {'ALLOWED' if allowed_all else 'BLOCKED'}")
print(f"require_all=False: {'ALLOWED' if allowed_any else 'BLOCKED'}")

Scenario: regime=trend_down (fails), drawdown=5% (passes)

require_all=True:  BLOCKED
require_all=False: ALLOWED


## 4. Signal Aggregation

Combine signals from multiple detectors using different voting strategies.

| Mode | Description |
|------|-------------|
| `MAJORITY` | Most common signal type wins (with min agreement threshold) |
| `WEIGHTED` | Weighted average by probability or custom weights |
| `UNANIMOUS` | All detectors must agree |
| `ANY` | Any non-NONE signal passes (highest probability wins) |
| `META_LABELING` | First detector provides direction, rest validate |

In [ ]:
# Create sample signals from different detectors
ts = datetime(2024, 1, 1)

# Detector 1: High confidence RISE
signals_1 = Signals(
    pl.DataFrame({
        "pair": ["BTCUSDT"],
        "timestamp": [ts],
        "signal_type": [SignalType.RISE.value],
        "signal": [1],
        "probability": [0.9],
    })
)

# Detector 2: Medium confidence RISE
signals_2 = Signals(
    pl.DataFrame({
        "pair": ["BTCUSDT"],
        "timestamp": [ts],
        "signal_type": [SignalType.RISE.value],
        "signal": [1],
        "probability": [0.7],
    })
)

# Detector 3: FALL signal (disagreement)
signals_3 = Signals(
    pl.DataFrame({
        "pair": ["BTCUSDT"],
        "timestamp": [ts],
        "signal_type": [SignalType.FALL.value],
        "signal": [-1],
        "probability": [0.6],
    })
)

print("Source signals:")
print(f"  Detector 1: RISE p=0.9")
print(f"  Detector 2: RISE p=0.7")
print(f"  Detector 3: FALL p=0.6")

Source signals:
  Detector 1: RISE p=0.9
  Detector 2: RISE p=0.7
  Detector 3: FALL p=0.6


### 4.1 Majority Voting

In [ ]:
agg = SignalAggregator(
    voting_mode=VotingMode.MAJORITY,
    min_agreement=0.5,  # Need 50%+ to agree
)

result = agg.aggregate([signals_1, signals_2, signals_3])
print(f"MAJORITY (min 50% agreement):")
print(f"  Signal: {result.value['signal_type'][0]}")
print(f"  Probability: {result.value['probability'][0]:.2f}")
print(f"  (2/3 = 67% agree on RISE)")

MAJORITY (min 50% agreement):
  Signal: rise
  Probability: 0.80
  (2/3 = 67% agree on RISE)


### 4.2 Weighted Voting

In [ ]:
# Weight by probability (default)
agg_prob = SignalAggregator(
    voting_mode=VotingMode.WEIGHTED,
    probability_threshold=0.5,
)

# Custom weights (favor detector 1)
agg_custom = SignalAggregator(
    voting_mode=VotingMode.WEIGHTED,
    weights=[3.0, 1.0, 1.0],  # Detector 1 has 3x weight
    probability_threshold=0.5,
)

result_prob = agg_prob.aggregate([signals_1, signals_2])
result_custom = agg_custom.aggregate([signals_1, signals_2])

print(f"WEIGHTED (probability):")
print(f"  Result: {result_prob.value['signal_type'][0]} p={result_prob.value['probability'][0]:.3f}")
print(f"  Expected: (0.9 + 0.7) / 2 = 0.80")

print(f"\nWEIGHTED (custom weights 3:1):")
print(f"  Result: {result_custom.value['signal_type'][0]} p={result_custom.value['probability'][0]:.3f}")
print(f"  Expected: (0.9*3 + 0.7*1) / 4 = 0.85")

WEIGHTED (probability):
  Result: rise p=0.800
  Expected: (0.9 + 0.7) / 2 = 0.80

WEIGHTED (custom weights 3:1):
  Result: rise p=0.850
  Expected: (0.9*3 + 0.7*1) / 4 = 0.85


### 4.3 Unanimous Voting

In [ ]:
agg = SignalAggregator(
    voting_mode=VotingMode.UNANIMOUS,
    probability_threshold=0.0,
)

# With agreement
result_agree = agg.aggregate([signals_1, signals_2])
print(f"UNANIMOUS (2 RISE signals): {result_agree.value.height} result(s)")

# With disagreement
result_disagree = agg.aggregate([signals_1, signals_3])
print(f"UNANIMOUS (RISE + FALL): {result_disagree.value.height} result(s) (filtered out)")

UNANIMOUS (2 RISE signals): 1 result(s)
UNANIMOUS (RISE + FALL): 0 result(s) (filtered out)


### 4.4 ANY Voting

In [ ]:
agg = SignalAggregator(voting_mode=VotingMode.ANY)

result = agg.aggregate([signals_1, signals_3])
print(f"ANY (RISE p=0.9 vs FALL p=0.6):")
print(f"  Winner: {result.value['signal_type'][0]} (highest probability)")
print(f"  Probability: {result.value['probability'][0]:.2f}")

ANY (RISE p=0.9 vs FALL p=0.6):
  Winner: rise (highest probability)
  Probability: 0.90


### 4.5 Meta-Labeling

First detector provides signal direction, subsequent detectors act as validators.

In [ ]:
# Validator confirms the signal
validator = Signals(
    pl.DataFrame({
        "pair": ["BTCUSDT"],
        "timestamp": [ts],
        "signal_type": [SignalType.RISE.value],  # Agrees
        "signal": [1],
        "probability": [0.85],  # High confidence
    })
)

agg = SignalAggregator(
    voting_mode=VotingMode.META_LABELING,
    probability_threshold=0.5,
)

result = agg.aggregate([signals_1, validator])
print(f"META_LABELING:")
print(f"  Detector: RISE p=0.9")
print(f"  Validator: RISE p=0.85")
print(f"  Combined: {result.value['signal_type'][0]} p={result.value['probability'][0]:.3f}")
print(f"  (0.9 * 0.85 = 0.765)")

META_LABELING:
  Detector: RISE p=0.9
  Validator: RISE p=0.85
  Combined: rise p=0.765
  (0.9 * 0.85 = 0.765)


## 5. Integration with SignalEntryRule

Position sizers and entry filters can be injected into `SignalEntryRule` for seamless integration.

In [ ]:
from signalflow.strategy.component.entry import SignalEntryRule

# Create entry rule with custom sizer and filters
entry_rule = SignalEntryRule(
    # Inject custom position sizer
    position_sizer=VolatilityTargetSizer(
        target_volatility=0.01,
        default_volatility_pct=0.02,
        max_fraction=0.15,
    ),
    
    # Inject entry filters
    entry_filters=CompositeEntryFilter(
        filters=[
            DrawdownFilter(max_drawdown=0.15),
            VolatilityFilter(max_volatility=0.05),
            TimeOfDayFilter(allowed_hours=list(range(8, 22))),
        ],
        require_all=True,
    ),
    
    # Fallback parameters (used if no sizer provided)
    base_position_size=500.0,
    min_probability=0.6,
    max_positions_per_pair=3,
    max_total_positions=10,
)

print("SignalEntryRule configured with:")
print(f"  Position sizer: {type(entry_rule.position_sizer).__name__}")
print(f"  Entry filters: {type(entry_rule.entry_filters).__name__}")

SignalEntryRule configured with:
  Position sizer: VolatilityTargetSizer
  Entry filters: CompositeEntryFilter


## 6. Grid Trading Example

Combine `MartingaleSizer` + `PriceDistanceFilter` for a complete grid trading setup.

In [ ]:
# Grid trading configuration
grid_sizer = MartingaleSizer(
    base_size=200.0,      # First entry: $200
    multiplier=1.5,       # Each level: 1.5x previous
    max_grid_levels=5,    # Max 5 entries
    max_notional=2000.0,  # Cap at $2000 per entry
)

grid_filter = CompositeEntryFilter(
    filters=[
        # Only add if price dropped 2%+ from last entry
        PriceDistanceFilter(min_distance_pct=0.02, direction_aware=True),
        # Only in favorable regimes
        RegimeFilter(),
        # Not during high volatility
        VolatilityFilter(max_volatility=0.05),
    ],
    require_all=True,
)

# Grid entry rule
grid_entry_rule = SignalEntryRule(
    position_sizer=grid_sizer,
    entry_filters=grid_filter,
    min_probability=0.5,
    max_positions_per_pair=5,  # Allow grid
)

print("Grid trading strategy configured:")
print(f"  Base size: $200")
print(f"  Grid levels: 5 max")
print(f"  Size progression: $200 -> $300 -> $450 -> $675 -> $1012")
print(f"  Min price distance: 2%")

Grid trading strategy configured:
  Base size: $200
  Grid levels: 5 max
  Size progression: $200 -> $300 -> $450 -> $675 -> $1012
  Min price distance: 2%


### Simulate Grid Entries

In [ ]:
# Simulate price drops and grid entries
entry_prices = [50000, 49000, 47500, 46000, 44500, 43000]

state_grid = create_test_state(50000.0)  # $50k capital
state_grid.runtime["regime"] = {"BTCUSDT": "trend_up"}
state_grid.runtime["atr"] = {"BTCUSDT": 1000.0}  # 2% vol

print("Simulating grid entries as price drops:\n")
total_invested = 0
positions = []

for i, price in enumerate(entry_prices):
    ctx = create_signal_context(
        signal_type="rise",
        probability=0.7,
        price=price,
    )
    
    # Check filter
    allowed, reason = grid_filter.allow_entry(ctx, state_grid, {"BTCUSDT": price})
    
    if allowed:
        # Compute size
        size = grid_sizer.compute_size(ctx, state_grid, {"BTCUSDT": price})
        
        if size > 0:
            qty = size / price
            total_invested += size
            
            # Add position to state
            pos = Position(
                pair="BTCUSDT",
                position_type=PositionType.LONG,
                entry_price=price,
                qty=qty,
                entry_time=datetime(2024, 1, 1, 10 + i, 0),
            )
            state_grid.portfolio.positions[pos.id] = pos
            positions.append((price, size, qty))
            
            print(f"Level {len(positions)}: BUY @ ${price:,} | Size: ${size:,.2f} | Qty: {qty:.6f} BTC")
        else:
            print(f"Level {len(positions)+1}: SKIP @ ${price:,} (max levels reached)")
    else:
        print(f"Attempt @ ${price:,}: BLOCKED - {reason}")

print(f"\n{'='*60}")
print(f"Total positions: {len(positions)}")
print(f"Total invested: ${total_invested:,.2f}")
avg_entry = sum(p[0] * p[2] for p in positions) / sum(p[2] for p in positions) if positions else 0
print(f"Average entry: ${avg_entry:,.2f}")

Simulating grid entries as price drops:

Level 1: BUY @ $50,000 | Size: $200.00 | Qty: 0.004000 BTC
Level 2: BUY @ $49,000 | Size: $300.00 | Qty: 0.006122 BTC
Level 3: BUY @ $47,500 | Size: $450.00 | Qty: 0.009474 BTC
Level 4: BUY @ $46,000 | Size: $675.00 | Qty: 0.014674 BTC
Level 5: BUY @ $44,500 | Size: $1,012.50 | Qty: 0.022753 BTC
Level 6: SKIP @ $43,000 (max levels reached)

Total positions: 5
Total invested: $2,637.50
Average entry: $46,253.38


## 7. Full Backtest Example

Complete example combining signal aggregation, filters, and sizing in a backtest.

In [ ]:
from pathlib import Path
from signalflow.data.raw_store import DuckDbRawStore
from signalflow.data.source import VirtualDataProvider
from signalflow.data import RawDataFactory
from signalflow.detector import ExampleSmaCrossDetector

# Generate synthetic data
PAIRS = ["BTCUSDT", "ETHUSDT"]
N_BARS = 5000
START = datetime(2025, 1, 1)

spot_store = DuckDbRawStore(db_path=Path("strategy_tutorial.duckdb"), timeframe="1m")

provider = VirtualDataProvider(
    store=spot_store,
    base_prices={"BTCUSDT": 50000.0, "ETHUSDT": 3000.0},
    volatility=0.003,
    seed=42,
)
provider.download(pairs=PAIRS, n_bars=N_BARS, start=START)

print(f"Generated {N_BARS} bars per pair")

2026-02-06 17:39:39.830 | INFO     | signalflow.data.raw_store.duckdb_stores:_ensure_tables:155 - Database initialized: strategy_tutorial.duckdb (data_type=spot, timeframe=1m)
2026-02-06 17:39:39.918 | DEBUG    | signalflow.data.raw_store.duckdb_stores:insert_klines:225 - Inserted 5,000 rows for BTCUSDT
2026-02-06 17:39:39.919 | INFO     | signalflow.data.source.virtual:download:255 - VirtualDataProvider: generated 5000 bars for BTCUSDT
2026-02-06 17:39:39.962 | DEBUG    | signalflow.data.raw_store.duckdb_stores:insert_klines:225 - Inserted 5,000 rows for ETHUSDT
2026-02-06 17:39:39.963 | INFO     | signalflow.data.source.virtual:download:255 - VirtualDataProvider: generated 5000 bars for ETHUSDT


Generated 5000 bars per pair


In [ ]:
# Load data
raw_data = RawDataFactory.from_duckdb_spot_store(
    spot_store_path=Path("strategy_tutorial.duckdb"),
    pairs=PAIRS,
    start=START,
    end=datetime(2025, 1, 4),
    data_types=["spot"],
)

raw_view = sf.RawDataView(raw=raw_data)
print(f"Loaded {raw_view.to_polars('spot').height} rows")

In [ ]:
# Create two detectors with different parameters
detector_fast = ExampleSmaCrossDetector(fast_period=10, slow_period=30)
detector_slow = ExampleSmaCrossDetector(fast_period=20, slow_period=50)

# Generate signals
signals_fast = detector_fast.run(raw_view)
signals_slow = detector_slow.run(raw_view)

print(f"Fast detector signals: {signals_fast.value.filter(pl.col('signal_type') != 'none').height}")
print(f"Slow detector signals: {signals_slow.value.filter(pl.col('signal_type') != 'none').height}")

In [ ]:
# Aggregate signals - require both detectors to agree
agg = SignalAggregator(
    voting_mode=VotingMode.UNANIMOUS,
    probability_threshold=0.0,
)

aggregated_signals = agg.aggregate([signals_fast, signals_slow])
active_signals = aggregated_signals.value.filter(pl.col("signal_type") != "none")

print(f"\nAfter UNANIMOUS aggregation: {active_signals.height} signals")
print(f"  Rise: {active_signals.filter(pl.col('signal_type') == 'rise').height}")
print(f"  Fall: {active_signals.filter(pl.col('signal_type') == 'fall').height}")

In [ ]:
from signalflow.strategy.broker import BacktestBroker
from signalflow.strategy.broker.executor import VirtualSpotExecutor
from signalflow.data.strategy_store import DuckDbStrategyStore
from signalflow.strategy.runner import OptimizedBacktestRunner
from signalflow.strategy.component.exit import TakeProfitStopLossExit
from signalflow.analytic.strategy import (
    TotalReturnMetric,
    DrawdownMetric,
    WinRateMetric,
)

INITIAL_CAPITAL = 10_000.0

# Strategy store
strategy_store = DuckDbStrategyStore("strategy_tutorial_results.duckdb")
strategy_store.init()

# Executor and broker
executor = VirtualSpotExecutor(fee_rate=0.001, slippage_pct=0.001)
broker = BacktestBroker(executor=executor, store=strategy_store)

# Entry rule with volatility-based sizing and filters
entry_rule = SignalEntryRule(
    position_sizer=VolatilityTargetSizer(
        target_volatility=0.015,
        default_volatility_pct=0.02,
        max_fraction=0.15,
    ),
    entry_filters=CompositeEntryFilter(
        filters=[
            DrawdownFilter(max_drawdown=0.10),
            TimeOfDayFilter(allowed_hours=list(range(6, 22))),
        ],
        require_all=True,
    ),
    min_probability=0.0,
    max_positions_per_pair=1,
    max_total_positions=5,
    allow_shorts=False,
)

# Exit rule
exit_rule = TakeProfitStopLossExit(
    take_profit_pct=0.02,
    stop_loss_pct=0.015,
)

# Metrics
metrics = [
    TotalReturnMetric(initial_capital=INITIAL_CAPITAL),
    DrawdownMetric(),
    WinRateMetric(),
]

print("Backtest configured")

In [ ]:
# Run backtest
runner = OptimizedBacktestRunner(
    strategy_id="strategy_tutorial",
    broker=broker,
    entry_rules=[entry_rule],
    exit_rules=[exit_rule],
    metrics=metrics,
    initial_capital=INITIAL_CAPITAL,
    data_key="spot",
)

# Use aggregated signals with probability column added
final_state = runner.run(raw_data, aggregated_signals)

In [ ]:
results = runner.get_results()

print("=" * 50)
print("BACKTEST RESULTS")
print("=" * 50)
print(f"  Initial Capital:  ${INITIAL_CAPITAL:,.2f}")
print(f"  Final Equity:     ${results.get('final_equity', 0):,.2f}")
print(f"  Total Return:     {results.get('final_return', 0) * 100:.2f}%")
print(f"  Max Drawdown:     {results.get('max_drawdown', 0) * 100:.2f}%")
print(f"  Win Rate:         {results.get('win_rate', 0) * 100:.1f}%")
print(f"  Total Trades:     {results.get('total_trades', 0)}")
print("=" * 50)

In [ ]:
# Cleanup
spot_store.close()
strategy_store.close()

# Optionally remove tutorial databases:
# Path("strategy_tutorial.duckdb").unlink(missing_ok=True)
# Path("strategy_tutorial_results.duckdb").unlink(missing_ok=True)

## Summary

This tutorial covered:

1. **Position Sizing Strategies**
   - `FixedFractionSizer`: Fixed % of equity
   - `SignalStrengthSizer`: Scale by probability
   - `KellyCriterionSizer`: Optimal f* formula
   - `VolatilityTargetSizer`: Inverse volatility
   - `RiskParitySizer`: Equal risk budget
   - `MartingaleSizer`: Grid/DCA scaling

2. **Entry Filters**
   - `RegimeFilter`: Match market regime
   - `VolatilityFilter`: Volatility range
   - `DrawdownFilter`: Pause on drawdown
   - `TimeOfDayFilter`: Trading hours
   - `PriceDistanceFilter`: Grid spacing
   - `SignalAccuracyFilter`: Detector performance
   - `CompositeEntryFilter`: AND/OR logic

3. **Signal Aggregation**
   - `MAJORITY`: Most common wins
   - `WEIGHTED`: Probability-weighted
   - `UNANIMOUS`: All must agree
   - `ANY`: Highest probability
   - `META_LABELING`: Detector + validator

4. **Integration**
   - Injectable components in `SignalEntryRule`
   - Grid trading example
   - Full backtest with all components